# 波の式のイメージ

$$
a(x, t) = A \sin(kx - \omega t + \phi) \hspace{30pt} (1) 
$$

ここで $A$ は振幅、$k$ は波数、$\omega$ は振動数、$\phi$ は位相を表します。

以下のコードでは波数、振動数、位相を変化させたときの波の形をアニメーションで描画します。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from matplotlib import cm
from IPython.display import HTML
import matplotlib

# アニメーションの埋め込み制限を増やす
matplotlib.rcParams['animation.embed_limit'] = 100  # MB

# 時空間での波の伝播可視化
# Y軸を時間軸として使用

# ハードコードされたパラメータ
A = 1.0                    # 振幅
k = 1.0                    # 波数
omega = 2.0                # 角周波数
phi = 0.0                  # 初期位相

# 導出される値
wavelength = 2 * np.pi / k
period = 2 * np.pi / omega
phase_velocity = omega / k

print(f"Wave parameters:")
print(f"  Wave number: k = {k:.2f} rad/m")
print(f"  Angular frequency: ω = {omega:.2f} rad/s")
print(f"  Wavelength: λ = {wavelength:.2f} m")
print(f"  Period: T = {period:.2f} s")
print(f"  Phase velocity: c = {phase_velocity:.2f} m/s")

# 空間と時間のグリッド
x = np.linspace(-5, 10, 200)
t = np.linspace(0, 10, 200)
X, T = np.meshgrid(x, t)

# 波動場の計算
# a(x,t) = A * sin(kx - ωt + φ)
Z = A * np.sin(k*X - omega*T + phi)

# 監視する2点
x1, x2 = 0, 2
idx1 = np.argmin(np.abs(x - x1))
idx2 = np.argmin(np.abs(x - x2))

# アニメーション版
fig_anim = plt.figure(figsize=(16, 10))

# アニメーション用のプロット
ax_contour = fig_anim.add_subplot(1, 2, 1)
ax_time = fig_anim.add_subplot(1, 2, 2)

# 固定された時間範囲
t_start = 0
t_end = 10
dt = 0.05
n_frames = int(t_end / dt)

# 固定されたグリッド（アニメーション全体で共通）
t_fixed = np.linspace(t_start, t_end, 200)
x_fixed = np.linspace(-5, 10, 200)
X_fixed, T_fixed = np.meshgrid(x_fixed, t_fixed)
Z_fixed = A * np.sin(k*X_fixed - omega*T_fixed + phi)

def init_anim():
    ax_contour.clear()
    ax_time.clear()
    return []

def animate(frame):
    ax_contour.clear()
    ax_time.clear()
    
    current_time = frame * dt
    
    # カラーコントアプロット（固定されたデータを使用）
    contour = ax_contour.contourf(X_fixed, T_fixed, Z_fixed, 
                                  levels=np.linspace(-1, 1, 21),
                                  cmap=cm.coolwarm, extend='both')
    
    # 等高線を追加
    ax_contour.contour(X_fixed, T_fixed, Z_fixed, 
                       levels=np.linspace(-1, 1, 11),
                       colors='black', alpha=0.3, linewidths=0.5)
    
    # 現在時刻を示す線（これだけが動く）
    ax_contour.axhline(y=current_time, color='green', linewidth=3, linestyle='-', label='Current time')
    
    # 監視点の位置を示す線
    ax_contour.axvline(x=x1, color='red', linewidth=1, linestyle=':', alpha=0.7)
    ax_contour.axvline(x=x2, color='blue', linewidth=1, linestyle=':', alpha=0.7)
    
    # 監視点での軌跡を強調
    ax_contour.plot(np.full_like(t_fixed, x1), t_fixed, 'w-', linewidth=2, alpha=0.5)
    ax_contour.plot(np.full_like(t_fixed, x2), t_fixed, 'w-', linewidth=2, alpha=0.5)
    
    ax_contour.set_xlabel('Position x [m]', fontsize=12)
    ax_contour.set_ylabel('Time t [s]', fontsize=12)
    ax_contour.set_title(f'Space-Time Wave Field (Contour Plot)', fontsize=14)
    ax_contour.set_xlim(-5, 10)
    ax_contour.set_ylim(t_start, t_end)
    
    # カラーバーを追加（初回のみ）
    if frame == 0:
        cbar = plt.colorbar(contour, ax=ax_contour)
        cbar.set_label('Amplitude', fontsize=12)
    
    # 右側のパネル：現在時刻での空間分布
    current_wave = A * np.sin(k*x - omega*current_time + phi)
    ax_time.plot(x, current_wave, 'k-', linewidth=2)
    
    # 監視点をマーク
    ax_time.plot(x1, A * np.sin(k*x1 - omega*current_time + phi), 'ro', markersize=10, label=f'x = {x1}')
    ax_time.plot(x2, A * np.sin(k*x2 - omega*current_time + phi), 'bo', markersize=10, label=f'x = {x2}')
    
    ax_time.set_xlabel('Position x [m]', fontsize=12)
    ax_time.set_ylabel('Amplitude', fontsize=12)
    ax_time.set_title(f'Spatial Distribution at t = {current_time:.2f} s', fontsize=14)
    ax_time.set_xlim(-5, 10)
    ax_time.set_ylim(-1.5, 1.5)
    ax_time.grid(True, alpha=0.3)
    ax_time.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax_time.axvline(x=x1, color='red', linestyle=':', alpha=0.5)
    ax_time.axvline(x=x2, color='blue', linestyle=':', alpha=0.5)
    ax_time.legend()
    
    # 波の情報を表示
    phase_at_x1 = (k*x1 - omega*current_time + phi) % (2*np.pi)
    phase_at_x2 = (k*x2 - omega*current_time + phi) % (2*np.pi)
    info_text = f'Phase at x={x1}: {phase_at_x1:.2f} rad\nPhase at x={x2}: {phase_at_x2:.2f} rad'
    ax_time.text(0.05, 0.95, info_text, transform=ax_time.transAxes, 
                verticalalignment='top', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    return []

# アニメーションの作成
anim = animation.FuncAnimation(fig_anim, animate, init_func=init_anim,
                              frames=n_frames, interval=50,
                              blit=False, repeat=True)

# HTMLとして表示
HTML(anim.to_jshtml())

In [2]:
# 静止画版：時空間波動場の可視化
fig_static = plt.figure(figsize=(16, 12))

# 1行目: 時空間コントアプロット
ax1 = fig_static.add_subplot(2, 2, (1, 2))

# カラーコントアプロット
contour = ax1.contourf(X, T, Z, levels=np.linspace(-1, 1, 21), cmap=cm.coolwarm, extend='both')
ax1.contour(X, T, Z, levels=np.linspace(-1, 1, 11), colors='black', alpha=0.3, linewidths=0.5)

# 監視点の位置
ax1.axvline(x=x1, color='red', linewidth=2, linestyle='--', alpha=0.8, label=f'x = {x1} m')
ax1.axvline(x=x2, color='blue', linewidth=2, linestyle='--', alpha=0.8, label=f'x = {x2} m')

# 波面（等位相線）を数本描画
for phase in np.arange(0, 10, wavelength/phase_velocity):
    x_phase = x
    t_phase = phase + k*x_phase/omega
    mask = (t_phase >= 0) & (t_phase <= 10)
    ax1.plot(x_phase[mask], t_phase[mask], 'k--', alpha=0.5, linewidth=1)

ax1.set_xlabel('Position x [m]', fontsize=12)
ax1.set_ylabel('Time t [s]', fontsize=12)
ax1.set_title('Space-Time Wave Field', fontsize=14)
ax1.set_xlim(-5, 10)
ax1.set_ylim(0, 10)
ax1.legend(loc='upper right')

# カラーバー
cbar = plt.colorbar(contour, ax=ax1)
cbar.set_label('Amplitude', fontsize=12)

# 2行目: 時間-振幅グラフ
ax2 = fig_static.add_subplot(2, 2, 3)
ax3 = fig_static.add_subplot(2, 2, 4)

# x=0での時間変化
ax2.plot(t, Z[:, idx1], 'r-', linewidth=2)
ax2.set_xlabel('Time t [s]', fontsize=12)
ax2.set_ylabel('Amplitude', fontsize=12)
ax2.set_title(f'Temporal Variation at x = {x1} m', fontsize=14)
ax2.set_ylim(-1.5, 1.5)
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)

# 特定の時刻をマーク
highlight_times = [0, period/4, period/2, 3*period/4, period]
for ht in highlight_times:
    if ht <= 10:
        ax2.axvline(x=ht, color='gray', linestyle=':', alpha=0.5)
        ax2.plot(ht, A * np.sin(k*x1 - omega*ht + phi), 'ko', markersize=8)

# x=2での時間変化
ax3.plot(t, Z[:, idx2], 'b-', linewidth=2)
ax3.set_xlabel('Time t [s]', fontsize=12)
ax3.set_ylabel('Amplitude', fontsize=12)
ax3.set_title(f'Temporal Variation at x = {x2} m', fontsize=14)
ax3.set_ylim(-1.5, 1.5)
ax3.grid(True, alpha=0.3)
ax3.axhline(y=0, color='k', linestyle='-', alpha=0.3)

# 特定の時刻をマーク
for ht in highlight_times:
    if ht <= 10:
        ax3.axvline(x=ht, color='gray', linestyle=':', alpha=0.5)
        ax3.plot(ht, A * np.sin(k*x2 - omega*ht + phi), 'ko', markersize=8)

# 位相差と時間遅延の情報
phase_diff = k * (x2 - x1)
time_diff = phase_diff / omega
ax3.text(0.05, 0.95, f'Phase difference: {phase_diff:.2f} rad\nTime delay: {time_diff:.2f} s', 
         transform=ax3.transAxes, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))